In [2]:
# Import modules.
import datetime
import numpy as np
import random
from random import shuffle
from random import randint
from geopy.distance import vincenty

from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Statistics
from pyevolve import Mutators
from pyevolve import Selectors

In [3]:
# Fitness function.
# It simply calculates the distance in kilometers of the current path.
def fitness(chromosome):
    score = 0
    for i in range(len(chromosome)):
        j = (i + 1) % len(chromosome)
        score += matrix[chromosome[i], chromosome[j]]
    return score

In [6]:
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Create a 1D list of tuples with latitude and longitude for each city.
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))
    
def getIndex(list_i, elem):    
    i = 0
    lisst = list_i.genomeList
    for value in lisst :
        if (value == elem):
            print('index', i)
            return i
        i = i + 1
        
def removeFrom(P, C):
    print('coucou remove')
    resP = P.clone()
    resul_gen = []
    for i in range(len(P)):
        found = False
        for j in range(len(C)):
            if(P[i] == C[j]):
                found = True
        if(not found):
            resul_gen += [P[i]]
    
    return  resul_gen

    
matrixx = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrixx[i, j] = dist
        
def dist_crit(i):
    tot = 0
    for j in range(len(matrixx[i])):
        tot += matrixx[i, j]
        
    return tot/2*(len(matrix[i]) - 1)
        
# Crossover Operator CX2
def G1DListCX2(genome, **args):
    global it
    it += 1
    C1   = None
    C2  = None
    P1 = args['mom'].clone()
    P2 = args['dad'].clone()
    print(args)
    if len(P1) == 1:
        Util.raiseException("The Binary String have one element, can't use the Two Point Crossover method !", TypeError)

    C1   = P2.clone()
    C2   = P2.clone()
    
    for i in range(14):
        C1[i] = 0
        C2[i] = 0
 
    ic1 = 0
    ic2 = 0
    C1[ic1] = P2[0]
    
    
    while(True):
        i0             = getIndex(P1, C1[ic1])
        temp           = P2[i0]
        i0             = getIndex(P1, temp)
       
        C2[ic2]  = P2[i0]
        ic1          +=  1
        if(ic1 >= 14):
            break
        i1            = getIndex(P1, C2[ic2])
        if(i1 == 0):
            P1.genomeList = removeFrom(P1, C2)
            P2.genomeList = removeFrom(P2, C1)
            print('P1 len', P1.genomeList)
            print('P2 len', P2.genomeList)
            C1[ic1] = P2[0]
            print('sister',C1.genomeList)
            print('brother',C2.genomeList)
        else:
            temp          = P2[i1]
            C1[ic1]  = temp
        ic2          += 1
            
        print('sister',C1.genomeList)
        print('brother',C2.genomeList)
        
    return C1, C2



def G1DListRAND(genome, **args):

    C1   = None
    C2  = None
    P1 = args['mom'].clone()
    P2 = args['dad'].clone()
    print(args)
    if len(P1) == 1:
        Util.raiseException("The Binary String have one element, can't use the Two Point Crossover method !", TypeError)

    C1   = P2.clone()
    C2   = P2.clone()
    
    for i in range(14):
        C1[i] = 0
        C2[i] = 0
 
    city_1 = [i for i in range(len(P1))]
    city_2 = [i for i in range(len(P1))]

    ic1 = 0
    ic2 = 0
    C1[0] = P1[randint(0, len(P1)-1)]
    C2[0] = P2[randint(0, len(P2)-1)]
    
    del city_1[city_1.index(C1[0])]
    del city_2[city_2.index(C2[0])]
    
    while(len(city_1) != 0):
        j1 = P1[getIndex(P2, C1[ic1]) + 1]
        j2 = P2[getIndex(P2, C1[ic1]) + 1]

        dj1 = matrix[C1[ic1], j1]
        dj2 = matrix[C1[ic1], j2]
        
        dmin = min(dj1, dj2)
        
        if(dmin < dist_crit(C1[ic1])):
            ic1 += 1
            if(dmin == dj1):
                C1[ic1] = j1
         #   elif(dmin == dj2):
            else:
                C1[ic1] = j2
 
            del city_1[city_1.index(C1[ic1])]
                
        else:
            ic1 += 1
            C1[ic1] = matrix[C1[ic1 - 1]].index(min(matrix[C1[ic1 - 1]]))
            del city_1[city_1.index(C1[ic1])]
            
            
    while(len(city_2) != 0):
        j1 = P1[getIndex(P2, C1[ic2]) + 1]
        j2 = P2[getIndex(P2, C1[ic2]) + 1]

        dj1 = matrix[C2[ic2], j1]
        dj2 = matrix[C2[ic2], j2]
        
        dmin = min(dj1, dj2)
        
        if(dmin < dist_crit(C2[ic2])):
            ic2 += 1
            if(dmin == dj1):
                C2[ic2] = j1
         #   elif(dmin == dj2):
            else:
                C2[ic2] = j2
 
            del city_2[city_2.index(C2[ic2])]
                
        else:
            ic2 += 1
            C2[ic2] = matrix[C2[ic2 - 1]].index(min(matrix[C1[ic2 - 1]]))
            del city_2[city_2.index(C2[ic2])]
        
    return C1, C2
        

SyntaxError: invalid syntax (<ipython-input-6-0cf894de6140>, line 7)

In [ ]:
# Callback called each time for each generation.
def tsp_callback(ga_engine):
    stats.append(ga.getStatistics().asTuple())
    return False

In [ ]:
# Initializator for the 1DList.
# Simply creates a list containing the cities indexes and shuffle it.
def list_initializator(genome, **args):
    cities = [i for i in range(len(coords))]
    random.shuffle(cities)
    genome.genomeList = cities
    #   genome.setInternalList(cities)


In [ ]:
# Save the GA statistics in `filename`.
def save_stats(filename, stats):
    header = 'rawMax;rawMin;rawAve;rawDev;rawVar;fitMax;fitMin;fitAve'
    np.savetxt(filename, stats, '%.2f', delimiter=';', header=header, comments='')
    print('Saved stats into ' + filename)

In [ ]:
# GPS position of each city.
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Create a 1D list of tuples with latitude and longitude for each city.
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# Create a distance matrix between each city.
matrix = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrix[i, j] = dist
        
# Constants.
POPULATION_SIZE = 100
MUTATION_RATE   = 0.01
CROSSOVER_RATE  = 0.9
GENERATIONS_NB  = 300
CROSSOVER_FUNC  = G1DListRAND
#CROSSOVER_FUNC  = Crossovers.G1DListCrossoverEdge

# Array for collecting statistics.
stats = []

# The chromosome is represented by a 1DList which contains the cities indexes [0,13]
genome = G1DList.G1DList(len(coords))
# Set fitness function.
genome.evaluator.set(fitness)
# Set crossover function.
genome.crossover.set(CROSSOVER_FUNC)
# Set initializator function.
genome.initializator.set(list_initializator)

# GA initialization.
ga = GSimpleGA.GSimpleGA(genome)

ga.setPopulationSize(POPULATION_SIZE)
ga.setMutationRate(MUTATION_RATE)

ga.setCrossoverRate(CROSSOVER_RATE)
# Elitism is important for TSP.
ga.selector.set(Selectors.GTournamentSelector)

ga.setElitism(True)
# We aim for minimization.
ga.setMinimax(Consts.minimaxType["minimize"])

# Number of generations
ga.setGenerations(GENERATIONS_NB)

# In case we want to monitor the evolution process
# execute the function current_best every generation
ga.stepCallback.set(tsp_callback)

ga.evolve(freq_stats=500)

# Final best solution
print(ga.bestIndividual())

# Save statistics.
if CROSSOVER_FUNC == Crossovers.G1DListCrossoverEdge:
    crossover_name = 'edge'
else:
    crossover_name = 'perso'
    
now = datetime.datetime.now()

save_stats('stats/' +
               crossover_name +
               '_pop' + str(POPULATION_SIZE) +
               '_gen' + str(GENERATIONS_NB) +
               '_' + now.strftime('%d%m%Y-%H%M') +
               '.csv',
           stats)

# Best path found: [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]
# For crossovers inspiration: https://www.hindawi.com/journals/cin/2017/7430125/